评：整个过程其实就是
1. 产出
2. 对产出进行评价，提出改进计划
3. 将产出和评价作为历史纪录，重复`1`，直到达到结束条件

In [ ]:
import './../../loadenv.mjs'

# Generate

In [2]:
import { getModel } from './../../utils.mjs'
import { ChatPromptTemplate, MessagesPlaceholder } from '@langchain/core/prompts'

const prompt = ChatPromptTemplate.fromMessages([
    [
        "system",
        `You are an essay assistant tasked with writing excellent 5-paragraph essays.
Generate the best essay possible for the user's request.  
If the user provides critique, respond with a revised version of your previous attempts.`,
    ],
    new MessagesPlaceholder('messages'),
])
const llm = getModel()
const essayGenerationChain = prompt.pipe(llm)

In [3]:
import { AIMessage, BaseMessage, HumanMessage } from '@langchain/core/messages'

let essay = ''
const request = new HumanMessage({
    content: 'Write an essay on why the little prince is relevant in modern childhood',
})
// for await (
//     const chunk of await essayGenerationChain.stream({
//         messages: [request],
//     })
// ) {
//     essay += chunk.content
// }
// essay

# Reflect

In [4]:
const reflectionPrompt = ChatPromptTemplate.fromMessages([
    [
        'system',
        `You are a teacher grading an essay submission.
Generate critique and recommendations for the user's submission.
Provide detailed recommendations, including requests for length, depth, style, etc.`
    ],
    new MessagesPlaceholder('messages'),
])
const reflect = reflectionPrompt.pipe(llm)

In [5]:
// let reflection = ''
// for await (
//     const chunk of await reflect.stream({
//         messages: [
//             request,
//             new HumanMessage({ content: essay }),
//         ]
//     })
// ) {
//     reflection += chunk.content
// }
// reflection

# Repeat

In [6]:
// let stream = await essayGenerationChain.stream({
//     messages: [
//         request,
//         new AIMessage({ content: essay }),
//         new HumanMessage({ content: reflection }),
//     ]
// })
// for await (const chunk of stream) {
//     console.log(chunk.content)
// }

# Define graph

In [7]:
import { END, MemorySaver, StateGraph, START, Annotation } from '@langchain/langgraph'

const State = Annotation.Root({
    messages: Annotation<BaseMessage[]>({
        reducer: (x, y) => x.concat(y),
    })
})

const generationNode = async (state: typeof State.State) => {
    const { messages } = state
    return {
        messages: [await essayGenerationChain.invoke({ messages })],
    }
}

const reflectionNode = async (state: typeof State.State) => {
    const { messages } = state
    const clsMap: { [key: string]: new (content: string) => BaseMessage } = {
        ai: HumanMessage,
        human: AIMessage,
    }
    const translated = [
        messages[0],
        ...messages.slice(1).map(msg => new clsMap[msg._getType()](msg.content.toString())),
    ]
    const res = await reflect.invoke({ messages: translated })
    return {
        messages: [new HumanMessage({ content: res.content })],
    }
}

In [ ]:
const workflow = new StateGraph(State)
    .addNode('generate', generationNode)
    .addNode('reflect', reflectionNode)
    .addEdge(START, 'generate')

const shouldContinue = (state: typeof State.State) => {
    const { messages } = state
    if (messages.length > 6) {
        return END
    }
    return 'reflect'
}

workflow
    .addConditionalEdges('generate', shouldContinue)
    .addEdge('reflect', 'generate')

const app = workflow.compile({ checkpointer: new MemorySaver() })

In [ ]:
import { printGraph } from './../../utils.mjs'
await printGraph(app.getGraph())

In [ ]:
const checkpointConfig = { configurable: { thread_id: 'my-thread' }}

let stream = await app.stream(
    {
        messages: [
            new HumanMessage({
                content: 'Generate an essay on the topicality of The Little Prince and its message in modern life',
            })
        ]
    },
    checkpointConfig,
)

for await (const event of stream) {
    for (const [key, _value] of Object.entries(event)) {
        console.log(`Event: ${key}`)
        console.log('\n------\n')
    }
}

In [ ]:
const snapshot = await app.getState(checkpointConfig)
console.log(
    snapshot.values.messages
        .map((msg: BaseMessage) => msg.content)
        .join('\n\n\n------------------\n\n\n')
)